In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from matplotlib.pyplot import hist

In [ ]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [ ]:
x = pd.read_stata("maindata.dta", convert_categoricals=False)
laws_csv = pd.read_csv("When_Were_Laws.csv")
laws_csv = laws_csv[np.logical_not(np.isnan(laws_csv["FIPS"]))]  # FIPS codes identify states
laws_csv = laws_csv.drop("State_Name", axis=1)  # Dropping as useless
laws_csv = laws_csv.rename({'FIPS': 'stfips'}, axis=1) 

# Merging
merged = pd.merge(laws_csv, x, on='stfips', how='outer')

In [ ]:
basic_merged = merged.copy()  # To allow for re-running 

basic_merged = basic_merged[basic_merged["a_age"] <= 25]  # Can be changed later, but for now useful I think
#age_subset = np.logical_and(np.greater_equal(basic_merged["a_age"],18), np.greater_equal(19,basic_merged["a_age"]))
# 17 <= age <= 21 (maybe should be like 22)
#basic_merged = basic_merged[age_subset]
#print(basic_merged.shape)

# Dropping states who were treated < 97 (i.e. they always had programs)
# This is following Callaway + Sant'anna, as we cannot meaningfully 
# do any inference using those states. Although we can compare them later as a 
# robustness check, which may be interesting
basic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("always")==False]  

# I also drop the never states, as they may be substantively different from others, although this can be relaxed later.
basic_merged = basic_merged.replace("never", "1000000") 
basic_merged["Year_Implemented"] = basic_merged["Year_Implemented"].astype(int)  # converting to intbasic_merged = basic_merged[basic_merged["Year_Implemented"].str.contains("never")==False]  # Only want to look at one for now. 

# As we are treating >19 as the never-treated group, we set their year implemented as 1000000 >> 1999
year_implemented_vector = basic_merged["Year_Implemented"].copy()
year_implemented_vector[basic_merged["under19"] == 0] = 1000000
basic_merged["group"] = year_implemented_vector  # Equals the year you were first treated. If >=19 then treated at t = infty

# Drop Arizona since they implemented late and later repealed policy
basic_merged = basic_merged[basic_merged["stfips"] != 5]

# Generating list of confounders of interest, these are not necessarily optimal. 
list_of_confounders = ["fownu18", "a_maritl", "female" , "povll"]#, "stfips"]
list_of_confounders += ["anykids", "disability", "collgrad", "hsgrad"] # coll + hs are extra for now. 

In [ ]:
def make_g_model():
  return RandomForestClassifier(random_state = 42, n_estimators=100, max_depth=10)

In [ ]:
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    """
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    """
    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
      X_train = X.loc[train_index]
      A_train = A.loc[train_index]
      g = make_model()
      g.fit(X_train, A_train)

      # get predictions for split
      predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]

    assert np.isnan(predictions).sum() == 0
    return predictions

In [ ]:
#overlap and g scores are calculated as the probabiliy that the indiviual is treated (under 19) in treated year given
#that you were either in that state in 1997 or never treated at all (over age of 19)
def calculate_g(treated_year):
    sub_merged = basic_merged.copy()
    
    #data of indiviuals 
    sub_merged = sub_merged[(sub_merged["group"] == treated_year) | (sub_merged["group"] == 1000000)]
    
    #Creating binary variablee
    treatment_bin = {treated_year: 1, 1000000: 0}
    sub_merged.group = [treatment_bin[item] for item in sub_merged.group]
    sub_merged = sub_merged.reset_index()
    
    treatment = sub_merged["group"]
    confounders = sub_merged[list_of_confounders]
    
    #Predicting g for a given year
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=10)
    
    return g
    

In [ ]:
g1 = calculate_g(1997)
#plotting the propensity scores to check overlap
hist(g1, density=True)

In [ ]:
#Checking the largest propensity score
g1.max()

In [ ]:
g2 = calculate_g(1998)
hist(g2, density=True)

In [ ]:
g2.max()

In [ ]:
g3 = calculate_g(1999)
hist(g3, density=True)

In [ ]:
g3.max()

In [ ]:
#function to loop through and different models and calculate the MSE and accuracy score for each one
def fit_g(treated_year, g_model):
    sub_merged = basic_merged.copy()
    sub_merged = sub_merged[(sub_merged["group"] == treated_year) | (sub_merged["group"] == 1000000)]
    
    treatment_bin = {treated_year: 1, 1000000: 0}
    sub_merged.group = [treatment_bin[item] for item in sub_merged.group]
    sub_merged = sub_merged.reset_index()
    
    treatment = sub_merged["group"]
    confounders = sub_merged[list_of_confounders]
    
    x_train, x_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)
    g_model.fit(x_train, a_train)
    a_pred = g_model.predict_proba(x_test)[:,1]
    
    #Calculating MSE and accuracy score
    test_ce=log_loss(a_test, a_pred)
    baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
    score = g_model.score(x_test,a_test)
    
    return test_ce, baseline_ce, score

In [ ]:
#list of models
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier

Rf_depth_2 = RandomForestClassifier(random_state = 42, n_estimators=100, max_depth=2)
Rf_depth_10 = RandomForestClassifier(random_state = 42, n_estimators=100, max_depth=10)
KNN = KNeighborsClassifier()
LogReg = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
XGBoost = sklearn.ensemble.GradientBoostingClassifier()

models = [Rf_depth_2, Rf_depth_10, KNN, LogReg, XGBoost]

In [ ]:
#function to find the fit of a g model for a given year
def fit_year(treated_year, models):
    fit_stat = pd.DataFrame()
    model_lst = ["Rf_depth2", "RF_depth10", "KNN", "LogReg", "XGBoost"]
    g_ce = []
    score = []
    baseline = []

    for model in models:
        x, y, z = fit_g(treated_year, model)
        g_ce.append(x)
        baseline.append(y)
        score.append(z)

    fit_stat["model"] = model_lst
    fit_stat["g_ce"] = g_ce
    fit_stat["baseline_ce"] = baseline
    fit_stat["accuracy_score"] = score

    return fit_stat

In [ ]:
fit_year(1997, models)

In [ ]:
fit_year(1999, models)

In [ ]:
#Only testing fit for 1998
sub_merged = basic_merged.copy()
sub_merged = sub_merged[(sub_merged["group"] == 1998) | (sub_merged["group"] == 1000000)]
    
treatment_bin = {1998: 1, 1000000: 0}
sub_merged.group = [treatment_bin[item] for item in sub_merged.group]
sub_merged = sub_merged.reset_index()
    
treatment = sub_merged["group"]
confounders = sub_merged[list_of_confounders]
    
x_train, x_test, a_train, a_test = train_test_split(confounders, treatment, test_size=0.2)

In [ ]:
def fit_g2(g_model):
    g_model.fit(x_train, a_train)
    a_pred = g_model.predict_proba(x_test)[:,1]
    
    test_ce=log_loss(a_test, a_pred)
    baseline_ce=log_loss(a_test, a_train.mean()*np.ones_like(a_test))
    score = g_model.score(x_test,a_test)
    
    return test_ce, baseline_ce, score

In [ ]:
fit_stat = pd.DataFrame()
model_lst = ["Rf_depth2", "RF_depth10", "KNN", "LogReg", "XGBoost"]
g_ce = []
score = []
baseline = []

for model in models:
    x, y, z = fit_g2(model)
    g_ce.append(x)
    baseline.append(y)
    score.append(z)

fit_stat["model"] = model_lst
fit_stat["g_ce"] = g_ce
fit_stat["baseline_ce"] = baseline
fit_stat["accuracy_score"] = score

fit_stat